# Código para optimizar probelmas de recursos


# Asignación de recursos categóricos

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def optimize_resource_allocation(resources, categories, value_matrix):
    """
    Optimiza la asignación de recursos a categorías utilizando programación dinámica.

    Parámetros:
    resources (int): Número total de recursos disponibles.
    categories (int): Número de categorías a las que se pueden asignar recursos.
    value_matrix (numpy array): Matriz de valores donde value_matrix[i][j] representa el valor
                                obtenido al asignar j+1 recursos a la categoría i+1.

    Retorna:
    tuple: Asignación óptima de recursos a cada categoría, el valor máximo obtenido y la tabla DP.
    """
    # Inicializar tabla de programación dinámica
    dp = np.zeros((categories + 1, resources + 1))

    # Llenar la tabla de programación dinámica
    for c in range(1, categories + 1):
        for r in range(1, resources + 1):
            max_value = 0
            for k in range(0, r + 1):  # Número de recursos asignados a la categoría c
                if c == 1:
                    max_value = max(max_value, value_matrix[c-1][k])
                else:
                    max_value = max(max_value, dp[c-1][r-k] + value_matrix[c-1][k])
            dp[c][r] = max_value

    # Obtener la asignación óptima
    assignment = [0] * categories
    remaining_resources = resources
    for c in range(categories, 0, -1):
        for k in range(0, remaining_resources + 1):
            if c == 1:
                if dp[c][remaining_resources] == value_matrix[c-1][k]:
                    assignment[c-1] = k
                    remaining_resources -= k
                    break
            else:
                if dp[c][remaining_resources] == dp[c-1][remaining_resources-k] + value_matrix[c-1][k]:
                    assignment[c-1] = k
                    remaining_resources -= k
                    break

    return assignment, dp[categories][resources], dp

# Ejemplo de uso
if __name__ == "__main__":
    # Definir el número de recursos y categorías
    num_resources = 5
    num_categories = 3

    # Definir la matriz de valores (años de vida adicionales por categoría para un número dado de recursos)
    value_matrix = np.array([
        [0, 45, 70, 90, 105, 120],   # Valores para el País 1
        [0, 20, 45, 75, 110, 150],   # Valores para el País 2
        [0, 50, 70, 80, 100, 130]    # Valores para el País 3
    ])

    # Optimizar la asignación de recursos
    optimal_assignment, max_value, dp = optimize_resource_allocation(num_resources, num_categories, value_matrix)


    # Mostrar los resultados
    print("La asignación óptima de recursos por país es:", optimal_assignment)
    print("El valor máximo obtenido es:", max_value)


La asignación óptima de recursos por categoría es: [3, 2, 1]
El valor máximo obtenido es: 157.0


# Asignación de recursos con probabilidades

In [5]:
import numpy as np

def minimize_failure_probability(num_resources, num_categories, failure_matrix):
    # Inicializar tabla de programación dinámica con un valor grande
    dp = np.full((num_categories + 1, num_resources + 1), np.inf)
    dp[0][0] = 1  # Probabilidad de fallo inicial es neutra

    # Llenar la tabla de programación dinámica
    for b in range(1, num_categories + 1):
        for m in range(num_resources + 1):
            for k in range(0, min(num_resources, m) + 1): 
                dp[b][m] = min(dp[b][m], dp[b-1][m-k] * failure_matrix[b-1][k])

    # Obtener la asignación óptima
    assignment = [0] * num_categories
    remaining_resources = num_resources  # Copia de num_resources
    for b in range(num_categories, 0, -1):
        for k in range(0, remaining_resources + 1):
            if dp[b][remaining_resources] == dp[b-1][remaining_resources-k] * failure_matrix[b-1][k]:
                assignment[b-1] = k
                remaining_resources -= k
                break

    return assignment, dp[num_categories][num_resources], dp


# Ejemplo de uso
if __name__ == "__main__":
    num_resources = 2
    num_categories = 3

    # Definir la matriz de probabilidades 
    failure_matrix = np.array([
        [0.4, 0.2, 0.15],  
        [0.6, 0.4, 0.2],  
        [0.8, 0.5, 0.3]   
    ])

    # Optimizar la asignación 
    optimal_assignment, min_failure, dp = minimize_failure_probability(num_resources, num_categories, failure_matrix)

    # Mostrar los resultados
    print("La asignación óptima :", optimal_assignment)
    print("La probabilidad mínima:", min_failure)


La asignación óptima : [1, 0, 1]
La probabilidad mínima: 0.06
